In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import random
import math
import statistics as st

In [ ]:
class Player:
    def __init__(self):
        pass
    
    def set_game(self, game):
        self.game = game
    
    def take_turn(self):
        rolls = [] 
        
        if self.will_roll(rolls):
            roll = self.game.get_roll()
            if roll in self.game.bust_numbers:
                return 0
            else:
                rolls.append(roll)
        return sum(rolls)
    
    def will_roll(self):
        return False
    

In [ ]:
class CautiousPlayer(Player):
    def will_roll(self,rolls):
        return len(rolls) <= 0

class RandomPlayer(Player):
    def will_roll(self,rolls):
        return random.randint(1,6) > 1

class HighRollingPlayer(Player):
    def will_roll(self,rolls):
        return sum(rolls) <= 50

            

In [ ]:
class PigGame:
    def __init__(self, players, turn_limit=math.inf, target=100, bust_numbers=[1]):
        self.players = players
        self.turn_limit = turn_limit
        def get_d6():
            return random.randint(1,6)
        self.get_roll = get_d6
        self.bust_numbers = bust_numbers
        self.target = target
    

        
    def play_a_round(self):
        self.scores = [0]*len(self.players)
        for i in range(self.turn_limit):
            for index, player in enumerate(self.players):
                self.scores[index] += player.take_turn()
            winners = [(player,score) for player, score in zip(self.players, self.scores) if score >= self.target]
            if len(winners) >0:
                return winners
        max_score = max(self.scores)
        return [(player,score) for player, score in zip(self.players, self.scores) if score == max_score]


In [ ]:
cautious = CautiousPlayer()
randy = RandomPlayer()
high_roller = HighRollingPlayer()
players = [cautious, randy, high_roller]
results = {}
for player in players:
    this_game = PigGame([player],7,math.inf)
    player.set_game(this_game)
    results[player] = []
    for i in range(10):
        returned = this_game.play_a_round()
        results[player].append(returned[0][1])
    del this_game

In [ ]:
print(results)

In [ ]:
results={}
for i in range(20):
    print("Limit:",i)
    totals = []
    for j in range(10000):
#         print("-Trial:",j+1)
        total = 0
        for k in range(7):
#             print("--Round:",k+1, "Total:",total)
            sub_total = 0
            for l in range(i):     
#             while sub_total < i:
                roll = random.randint(1,6)
                if roll == 1:
                    sub_total =0
#                     print("---Roll#:", l+1, "SubTotal:",sub_total, "Roll:",roll)
                    break
                else:
                    sub_total += roll
#                     print("---Roll#:", l+1, "SubTotal:",sub_total, "Roll:",roll)
            total += sub_total
#             print("--End of Round",k+1, "Final:",total)
        totals.append(total)
    results[i] = totals



In [ ]:
results


In [ ]:
avgs = {}
for i in results.keys():
    avgs[i]= st.median(results[i])

In [ ]:
print(avgs)


In [ ]:
results[5]
